# Data Cleaning and Preparation

During the course of doing data analysis and modeling, a significant amount of time is spent on data preparation: loading, cleaning, transforming, and rearranging. Such tasks are often reported to take up **80%** or more of an analyst's time. Sometimes the way that data is stored in files or databases is not in the right format for a particular task. Many researchers choose to do ad hoc processing of data from one form to another using a general-purpose programming language, like Python, Perl, R, or Java, or Unix text-processing tools like sed or awk. Fortunately, pandas, along with the built-in Python language features, provides you with a high-level, flexible, and fast set of tools to enable you to manipulate data into the right form.

In this chapter I discuss tools for missing data, duplicate data, string manipulation, and some other analytical data transformations. In the next chapter, I focus on combining and rearranging datasets in various ways.

## Handling Missing Data

Missing data occurs commonly in many data analysis applications. One of the goals of pandas is to make working with missing data as painless as possible. For example, all of the descriptive statistics on pandas objects exclude missing data by default.

The way that missing data is represented in pandas objects is somewhat imperfect, but it is sufficient for most real-world use. For data with float64 dtype, pandas uses the floating-point value NaN (Not a Number) to represent missing data.

In [ ]:
import numpy as np
import pandas as pd
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 25
pd.options.display.max_columns = 20
pd.options.display.max_colwidth = 82
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc("figure", figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

In [ ]:
import numpy as np
import pandas as pd

We call this a sentinel value: when present, it indicates a missing (or null) value:

In [ ]:
float_data = pd.Series([1.2, -3.5, np.nan, 0])
float_data

The isna method gives us a Boolean Series with True where values are null:

In [ ]:
float_data.isna()

In pandas, we've adopted a convention used in the R programming language by referring to missing data as NA, which stands for not available. In statistics applications, NA data may either be data that does not exist or that exists but was not observed (through problems with data collection, for example). When cleaning up data for analysis, it is often important to do analysis on the missing data itself to identify data collection problems or potential biases in the data caused by missing data.

The built-in Python None value is also treated as NA:

In [ ]:
string_data = pd.Series(["aardvark", np.nan, None, "avocado"])
string_data


In [ ]:
string_data.isna()


In [ ]:
float_data = pd.Series([1, 2, None], dtype='float64')
float_data

In [ ]:

float_data.notna()

The pandas project has attempted to make working with missing data consistent across data types. Functions like pandas.isna abstract away many of the annoying details. See Table 7.1 for a list of some functions related to missing data handling.

# ![title](i71.png)

### Filtering Out Missing Data
There are a few ways to filter out missing data. While you always have the option to do it by hand using pandas.isna and Boolean indexing, dropna can be helpful. On a Series, it returns the Series with only the nonnull data and index values:

In [ ]:
data = pd.Series([1, np.nan, 3.5, np.nan, 7])
data

In [ ]:
data.dropna()

This is the same thing as doing:

In [ ]:
data[data.notna()]

With DataFrame objects, there are different ways to remove missing data. You may want to **drop rows or columns that are all NA, or only those rows or columns containing any NAs at all**. dropna by default drops any row containing a missing value:

In [ ]:
data = pd.DataFrame([[1., 6.5, 3.], [1., np.nan, np.nan],
                     [np.nan, np.nan, np.nan], [np.nan, 6.5, 3.]])
data

In [ ]:
data.dropna()

Passing how="all" will drop only rows that are all NA:

In [ ]:
data.dropna(how="all")

Keep in mind that these functions return new objects by default and do not modify the contents of the original object.

To drop columns in the same way, pass axis="columns":

In [ ]:
data[4] = np.nan
data


In [ ]:
data.dropna(axis="columns", how="all")

Suppose you want to keep only rows containing at most a certain number of missing observations. You can indicate this with the thresh argument:

In [ ]:
df = pd.DataFrame(np.random.standard_normal((7, 3)))
df

In [ ]:
df.iloc[:4, 1] = np.nan
df.iloc[:2, 2] = np.nan
df


In [ ]:
df.dropna()


In [ ]:
df.dropna(thresh=2)

### Filling In Missing Data
Rather than filtering out missing data (and potentially discarding other data along with it), you may want to fill in the “holes” in any number of ways. For most purposes, the fillna method is the workhorse function to use. Calling fillna with a constant replaces missing values with that value:

In [ ]:
df

In [ ]:
df.fillna(0)

Calling fillna with a dictionary, you can use a different fill value for each column:

In [ ]:
df.fillna({1: 0.5, 2: 0})

The same interpolation methods available for reindexing (see Table 5.3) can be used with fillna:

In [ ]:
df = pd.DataFrame(np.random.standard_normal((6, 3)))
df.iloc[2:, 1] = np.nan
df.iloc[4:, 2] = np.nan
df


In [ ]:
df.fillna(method="ffill")


In [ ]:
df.fillna(method="ffill", limit=2)

With fillna you can do lots of other things such as simple data imputation using the median or mean statistics:

In [ ]:
data = pd.Series([1., np.nan, 3.5, np.nan, 7])
data.fillna(data.mean())

See Table 7.2 for a reference on fillna function arguments.

# ![title](i72.png)

## Data Transformation
So far in this chapter we’ve been concerned with handling missing data. Filtering, cleaning, and other transformations are another class of important operations.

### Removing Duplicates
Duplicate rows may be found in a DataFrame for any number of reasons. Here is an example:

In [1]:
import pandas as pd
import numpy as np

data = pd.DataFrame({"k1": ["one", "two"] * 3 + ["two"],
                     "k2": [1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


The DataFrame method duplicated returns a Boolean Series indicating whether each row is a duplicate (its column values are exactly equal to those in an earlier row) or not:

In [2]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

Relatedly, drop_duplicates returns a DataFrame with rows where the duplicated array is False filtered out:

In [3]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


Both methods by default consider all of the columns; alternatively, you can specify any subset of them to detect duplicates. Suppose we had an additional column of values and wanted to filter duplicates based only on the "k1" column:

In [4]:
data["v1"] = range(7)
data


,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [5]:
data.drop_duplicates(subset=["k1"])

,k1,k2,v1
0,one,1,0
1,two,1,1


In [6]:
data.drop_duplicates(["k1", "k2"])

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5


duplicated and drop_duplicates by default keep the first observed value combination. Passing keep="last" will return the last one:

In [7]:
data.drop_duplicates(["k1", "k2"], keep="last")

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


### Transforming Data Using a Function or Mapping
For many datasets, you may wish to perform some transformation based on the values in an array, Series, or column in a DataFrame. Consider the following hypothetical data collected about various kinds of meat:

In [8]:
data = pd.DataFrame({"food": ["bacon", "pulled pork", "bacon",
                              "pastrami", "corned beef", "bacon",
                              "pastrami", "honey ham", "nova lox"],
                     "ounces": [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,pastrami,6.0
4,corned beef,7.5
5,bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


Suppose you wanted to add a column indicating the type of animal that each food came from. Let’s write down a mapping of each distinct meat type to the kind of animal:

In [9]:
meat_to_animal = {
  "bacon": "pig",
  "pulled pork": "pig",
  "pastrami": "cow",
  "corned beef": "cow",
  "honey ham": "pig",
  "nova lox": "salmon"
}

The map method on a Series (also discussed in Ch 5.2.5: Function Application and Mapping) accepts a function or dictionary-like object containing a mapping to do the transformation of values:

In [10]:
data["animal"] = data["food"].map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,pastrami,6.0,cow
4,corned beef,7.5,cow
5,bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


We could also have passed a function that does all the work:



In [11]:
def get_animal(x):
    return meat_to_animal[x]
data["new_animal"] = data["food"].map(get_animal)
data

,food,ounces,animal,new_animal
0,bacon,4.0,pig,pig
1,pulled pork,3.0,pig,pig
2,bacon,12.0,pig,pig
3,pastrami,6.0,cow,cow
4,corned beef,7.5,cow,cow
5,bacon,8.0,pig,pig
6,pastrami,3.0,cow,cow
7,honey ham,5.0,pig,pig
8,nova lox,6.0,salmon,salmon


Using map is a convenient way to perform element-wise transformations and other data cleaning-related operations.

### Replacing Values
Filling in missing data with the fillna method is a special case of more general value replacement. As you've already seen, map can be used to modify a subset of values in an object, but replace provides a simpler and more flexible way to do so. Let’s consider this Series:

In [12]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

The -999 values might be sentinel values for missing data. To replace these with NA values that pandas understands, we can use replace, producing a new Series:

In [13]:
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

If you want to replace multiple values at once, you instead pass a list and then the substitute value:



In [14]:
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

To use a different replacement for each value, pass a list of substitutes:



In [15]:
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

The argument passed can also be a dictionary:

In [16]:
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

### Renaming Axis Indexes
Like values in a Series, axis labels can be similarly transformed by a function or mapping of some form to produce new, differently labeled objects. You can also modify the axes in place without creating a new data structure. Here’s a simple example:

In [17]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=["Ohio", "Colorado", "New York"],
                    columns=["one", "two", "three", "four"])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [18]:
data.index[1] = "COLO"

TypeError: Index does not support mutable operations

Like a Series, the axis indexes have a map method:

In [19]:
def transform(x):
    return x[:4].upper()

data.index = data.index.map(transform)
data.index

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

In [20]:
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


Notably, rename can be used in conjunction with a dictionary-like object, providing new values for a subset of the axis labels:

In [21]:
data.rename(index={"OHIO": "INDIANA"}, columns={"three": "peekaboo"})

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


rename saves you from the chore of copying the DataFrame manually and assigning new values to its index and columns attributes.

## Discretization and Binning
Continuous data is often discretized or otherwise separated into “bins” for analysis. Suppose you have data about a group of people in a study, and you want to group them into discrete age buckets:

In [22]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

Let’s divide these into bins of 18 to 25, 26 to 35, 36 to 60, and finally 61 and older. To do so, you have to use pandas.cut:

In [23]:
bins = [18, 25, 35, 60, 100]
age_categories = pd.cut(ages, bins)

In [24]:
age_categories

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64, right]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

The object pandas returns is a special Categorical object. The output you see describes the bins computed by pandas.cut. Each bin is identified by a special (unique to pandas) interval value type containing the lower and upper limit of each bin:

In [26]:
print(ages)
age_categories.codes

[20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]


array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [27]:
age_categories.categories


IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]], dtype='interval[int64, right]')

In [28]:
age_categories.categories[0]


Interval(18, 25, closed='right')

In [34]:
pd.value_counts(age_categories)

Youth         5
YoungAdult    3
MiddleAged    3
Senior        1
dtype: int64

Note that pd.value_counts(categories) are the bin counts for the result of pandas.cut.

In the string representation of an interval, a parenthesis means that the side is open (exclusive), while the square bracket means it is closed (inclusive). You can change which side is closed by passing right=False:

In [30]:
pd.cut(ages, bins, right=False)

[[18, 25), [18, 25), [25, 35), [25, 35), [18, 25), ..., [25, 35), [60, 100), [35, 60), [35, 60), [25, 35)]
Length: 12
Categories (4, interval[int64, left]): [[18, 25) < [25, 35) < [35, 60) < [60, 100)]

You can override the default interval-based bin labeling by passing a list or array to the labels option:



In [33]:
group_names = ["Youth", "YoungAdult", "MiddleAged", "Senior"]
age_categories = pd.cut(ages, bins, labels=group_names)

If you pass an integer number of bins to pandas.cut instead of explicit bin edges, it will compute equal-length bins based on the minimum and maximum values in the data. Consider the case of some uniformly distributed data chopped into fourths:

In [43]:
data = np.random.uniform(size=100)
print(data)
tmp = pd.cut(data, 4, precision=2)

[0.2611692  0.55980691 0.2612824  0.79117062 0.10733192 0.59439592
 0.96568864 0.27598683 0.25898757 0.94691385 0.81748813 0.11018104
 0.63577634 0.63753491 0.82629606 0.81143799 0.44528788 0.40905607
 0.15527154 0.4946043  0.89634966 0.43713906 0.75573358 0.02437412
 0.9857106  0.22805371 0.41655015 0.41503583 0.9643087  0.18518552
 0.72909955 0.81049438 0.3254975  0.75319909 0.32451349 0.69234509
 0.21400701 0.75616769 0.59721261 0.16665336 0.80690857 0.88042057
 0.47531692 0.54411367 0.87451886 0.523621   0.61694097 0.25590082
 0.01482531 0.27601125 0.57049536 0.48343732 0.61526792 0.9988538
 0.80702475 0.69083246 0.51911613 0.2605364  0.03249149 0.19435665
 0.2861733  0.65966972 0.16110696 0.93699063 0.73288024 0.33936906
 0.90752924 0.27583723 0.63314798 0.16944316 0.03666339 0.83148923
 0.63974318 0.9868123  0.70543977 0.00950737 0.8844285  0.04936046
 0.01632467 0.12686794 0.50802489 0.28629102 0.68732769 0.5151992
 0.60163213 0.40725425 0.53077855 0.49528158 0.1502101  0.708202

In [44]:
pd.value_counts(tmp)

(0.26, 0.5]       26
(0.75, 1.0]       26
(0.5, 0.75]       25
(0.0085, 0.26]    23
dtype: int64

The precision=2 option limits the decimal precision to two digits.

A closely related function, pandas.qcut, bins the data based on sample quantiles. Depending on the distribution of the data, using pandas.cut will not usually result in each bin having the same number of data points. Since pandas.qcut uses sample quantiles instead, you will obtain roughly equally sized bins:

In [50]:
data = np.random.standard_normal(1000)
data

array([-0.93129463,  0.72250843,  1.08221862, -0.3825675 , -1.34706842,
        0.09704367,  0.6913845 , -0.12793874,  0.38966957,  0.22659567,
       -1.09551777,  0.08806441,  0.11468203, -0.70923533, -0.51812764,
       -0.61804266, -0.68721159, -0.11164092, -0.42767754,  0.63008309,
       -0.28199768,  0.92221409, -0.02338592, -0.2496411 ,  2.03946924,
       -1.2626887 , -1.59538931,  0.5626561 ,  0.8266608 ,  0.87451534,
        1.14246702, -0.08619854,  0.0752616 ,  0.120213  ,  2.32192627,
       -0.05476711,  0.06569372,  0.10225788, -1.2291934 , -0.58800495,
        0.57969756, -0.71267063,  0.81189769,  0.53242331,  0.34347522,
       -0.76659927, -0.7714954 ,  1.3402298 , -0.76502223,  1.0435165 ,
        0.73080262, -0.10201365,  0.56033318, -0.83808631, -1.12921549,
        0.44494527,  0.49748111,  1.27923779, -0.95397421,  0.94577752,
        0.0422711 , -0.25711383,  0.12687389,  0.14625166, -0.38209687,
       -1.60740965,  0.31124746,  0.58697331,  0.54461741,  0.76

In [53]:
quartiles = pd.qcut(data, 4, precision=2)
quartiles

[(-3.03, -0.56], (0.088, 0.76], (0.76, 3.02], (-0.56, 0.088], (-3.03, -0.56], ..., (0.088, 0.76], (0.088, 0.76], (0.088, 0.76], (-3.03, -0.56], (0.088, 0.76]]
Length: 1000
Categories (4, interval[float64, right]): [(-3.03, -0.56] < (-0.56, 0.088] < (0.088, 0.76] < (0.76, 3.02]]

In [54]:
pd.value_counts(quartiles)

(-3.03, -0.56]    250
(-0.56, 0.088]    250
(0.088, 0.76]     250
(0.76, 3.02]      250
dtype: int64

Similar to pandas.cut, you can pass your own quantiles (numbers between 0 and 1, inclusive):



In [55]:
pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.]).value_counts()

(-3.022, -1.2]     100
(-1.2, 0.0884]     400
(0.0884, 1.305]    400
(1.305, 3.015]     100
dtype: int64

### Detecting and Filtering Outliers
Filtering or transforming outliers is largely a matter of applying array operations. Consider a DataFrame with some normally distributed data:

In [57]:
data = pd.DataFrame(np.random.standard_normal((1000, 4)))
data

,0,1,2,3
0,-0.023162,-1.301293,1.096440,0.653144
1,-0.044889,-0.066222,-1.159135,-1.292397
2,-1.043845,-0.307623,2.058510,1.010988
3,-0.170137,0.882518,0.995050,-0.556402
4,0.570255,-1.533676,-0.484893,-0.145494
...,...,...,...,...
995,-0.270966,-0.142053,0.795025,-1.948603
996,-0.743478,-0.708117,-1.623002,-0.209270
997,-0.685003,-0.527841,-0.147888,0.434633
998,0.223088,-0.203822,1.760582,1.160005


In [58]:
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.006739,0.017613,0.033042,-0.021498
std,0.984282,0.998441,1.010689,1.017868
min,-2.954733,-2.919253,-3.474465,-3.305112
25%,-0.662335,-0.665213,-0.666030,-0.678225
50%,-0.023320,0.017976,0.038329,-0.018848
75%,0.637123,0.701819,0.735772,0.639305
max,4.021066,3.632101,3.422138,3.524414


Suppose you wanted to find values in one of the columns exceeding 3 in absolute value:



In [59]:
col = data[2]
col[col.abs() > 3]

6      3.024314
118    3.070639
465   -3.474465
495   -3.025909
883    3.054584
992    3.422138
Name: 2, dtype: float64

To select all rows having a value exceeding 3 or –3, you can use the any method on a Boolean DataFrame:

In [68]:
data[(data.abs() >= 3).any(axis="columns")]

,0,1,2,3
6,0.762202,-0.874882,3.000000,1.763858
15,0.518683,3.000000,0.347959,-0.447180
118,-0.058114,-0.756396,3.000000,-0.264394
142,0.420385,-0.300385,-0.081170,3.000000
253,0.984879,0.919705,-0.040371,3.000000
465,0.989295,-0.529669,-3.000000,-0.083645
495,2.195832,0.502410,-3.000000,0.326798
573,0.606121,-0.435224,-0.974815,-3.000000
793,3.000000,0.245242,-0.764892,1.771752
803,0.943621,3.000000,-0.369115,0.031297


The parentheses around data.abs() > 3 are necessary in order to call the any method on the result of the comparison operation.

Values can be set based on these criteria. Here is code to cap values outside the interval –3 to 3:

In [69]:
data[data.abs() > 3] = np.sign(data) * 3
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.007760,0.016844,0.032971,-0.021847
std,0.980622,0.995929,1.007296,1.014837
min,-2.954733,-2.919253,-3.000000,-3.000000
25%,-0.662335,-0.665213,-0.666030,-0.678225
50%,-0.023320,0.017976,0.038329,-0.018848
75%,0.637123,0.701819,0.735772,0.639305
max,3.000000,3.000000,3.000000,3.000000


The statement np.sign(data) produces 1 and –1 values based on whether the values in data are positive or negative:

In [64]:
data.head(10)

,0,1,2,3
0,-0.023162,-1.301293,1.096440,0.653144
1,-0.044889,-0.066222,-1.159135,-1.292397
2,-1.043845,-0.307623,2.058510,1.010988
3,-0.170137,0.882518,0.995050,-0.556402
4,0.570255,-1.533676,-0.484893,-0.145494
5,-0.619316,1.077459,0.006327,0.182589
6,0.762202,-0.874882,3.024314,1.763858
7,-0.346724,-0.301732,-0.086278,0.380929
8,0.718623,2.105156,-0.962351,0.714221
9,-1.286487,0.086462,0.383660,-0.669776


In [63]:
np.sign(data).head(10)

,0,1,2,3
0,-1.0,-1.0,1.0,1.0
1,-1.0,-1.0,-1.0,-1.0
2,-1.0,-1.0,1.0,1.0
3,-1.0,1.0,1.0,-1.0
4,1.0,-1.0,-1.0,-1.0
5,-1.0,1.0,1.0,1.0
6,1.0,-1.0,1.0,1.0
7,-1.0,-1.0,-1.0,1.0
8,1.0,1.0,-1.0,1.0
9,-1.0,1.0,1.0,-1.0


## Permutation and Random Sampling
Permuting (randomly reordering) a Series or the rows in a DataFrame is possible using the numpy.random.permutation function. Calling permutation with the length of the axis you want to permute produces an array of integers indicating the new ordering:

In [70]:
df = pd.DataFrame(np.arange(5 * 7).reshape((5, 7)))
df

,0,1,2,3,4,5,6
0,0,1,2,3,4,5,6
1,7,8,9,10,11,12,13
2,14,15,16,17,18,19,20
3,21,22,23,24,25,26,27
4,28,29,30,31,32,33,34


In [72]:
sampler = np.random.permutation(5)
sampler

array([4, 2, 1, 0, 3])

That array can then be used in iloc-based indexing or the equivalent take function:

In [73]:
df.take(sampler)


,0,1,2,3,4,5,6
4,28,29,30,31,32,33,34
2,14,15,16,17,18,19,20
1,7,8,9,10,11,12,13
0,0,1,2,3,4,5,6
3,21,22,23,24,25,26,27


In [74]:
df.iloc[sampler]

,0,1,2,3,4,5,6
4,28,29,30,31,32,33,34
2,14,15,16,17,18,19,20
1,7,8,9,10,11,12,13
0,0,1,2,3,4,5,6
3,21,22,23,24,25,26,27


By invoking take with axis="columns", we could also select a permutation of the columns:



In [76]:
column_sampler = np.random.permutation(7)
column_sampler
df.take(column_sampler, axis="columns")

,4,6,2,0,1,5,3
0,4,6,2,0,1,5,3
1,11,13,9,7,8,12,10
2,18,20,16,14,15,19,17
3,25,27,23,21,22,26,24
4,32,34,30,28,29,33,31


To select a random subset without replacement (the same row cannot appear twice), you can use the sample method on Series and DataFrame:

In [77]:
df.sample(n=3)

,0,1,2,3,4,5,6
4,28,29,30,31,32,33,34
0,0,1,2,3,4,5,6
3,21,22,23,24,25,26,27


To generate a sample with replacement (to allow repeat choices), pass replace=True to sample:



In [78]:
choices = pd.Series([5, 7, -1, 6, 4])
choices

0    5
1    7
2   -1
3    6
4    4
dtype: int64

In [82]:
choices.sample(n=10, replace=True)

1    7
1    7
4    4
1    7
1    7
2   -1
0    5
0    5
0    5
2   -1
dtype: int64

### Computing Indicator/Dummy Variables
Another type of transformation for statistical modeling or machine learning applications is converting a categorical variable into a dummy or indicator matrix. If a column in a DataFrame has k distinct values, you would derive a matrix or DataFrame with k columns containing all 1s and 0s. pandas has a pandas.get_dummies function for doing this, though you could also devise one yourself. Let’s consider an example DataFrame:

In [2]:
import pandas as pd
import numpy as np
df = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "b"],
                   "data1": range(6)})
df

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [6]:
pd.get_dummies(df["key"], dtype=float)

,a,b,c
0,0.0,1.0,0.0
1,0.0,1.0,0.0
2,1.0,0.0,0.0
3,0.0,0.0,1.0
4,1.0,0.0,0.0
5,0.0,1.0,0.0


Here I passed dtype=float to change the output type from boolean (the default in more recent versions of pandas) to floating point.

In some cases, you may want to add a prefix to the columns in the indicator DataFrame, which can then be merged with the other data. pandas.get_dummies has a prefix argument for doing this:

In [7]:
dummies = pd.get_dummies(df["key"], prefix="key", dtype=float)
dummies

,key_a,key_b,key_c
0,0.0,1.0,0.0
1,0.0,1.0,0.0
2,1.0,0.0,0.0
3,0.0,0.0,1.0
4,1.0,0.0,0.0
5,0.0,1.0,0.0


In [8]:
df_with_dummy = df[["data1"]].join(dummies)
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0.0,1.0,0.0
1,1,0.0,1.0,0.0
2,2,1.0,0.0,0.0
3,3,0.0,0.0,1.0
4,4,1.0,0.0,0.0
5,5,0.0,1.0,0.0


In [9]:
pd.concat((df["data1"], dummies), axis=1)

,data1,key_a,key_b,key_c
0,0,0.0,1.0,0.0
1,1,0.0,1.0,0.0
2,2,1.0,0.0,0.0
3,3,0.0,0.0,1.0
4,4,1.0,0.0,0.0
5,5,0.0,1.0,0.0


The DataFrame.join method will be explained in more detail in the next chapter.

If a row in a DataFrame belongs to multiple categories, we have to use a different approach to create the dummy variables. Let’s look at the MovieLens 1M dataset, which is investigated in more detail in Ch 13: Data Analysis Examples:

In [10]:
mnames = ["movie_id", "title", "genres"]
movies = pd.read_table("datasets/movielens/movies.dat", sep="::",
                       header=None, names=mnames, engine="python")
movies.head(10)

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


pandas has implemented a special Series method str.get_dummies (methods that start with str. are discussed in more detail later in String Manipulation) that handles this scenario of multiple group membership encoded as a delimited string:

In [11]:
dummies = movies["genres"].str.get_dummies("|")
dummies.iloc[:10]

,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
5,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
6,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
7,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


Then, as before, you can combine this with movies while adding a "Genre_" to the column names in the dummies DataFrame with the add_prefix method:

In [21]:
movies_windic = movies.join(dummies.add_prefix("Genre_"))
movies_windic.head(10)

,movie_id,title,genres,Genre_Action,Genre_Adventure,Genre_Animation,Genre_Children's,Genre_Comedy,Genre_Crime,Genre_Documentary,...,Genre_Fantasy,Genre_Film-Noir,Genre_Horror,Genre_Musical,Genre_Mystery,Genre_Romance,Genre_Sci-Fi,Genre_Thriller,Genre_War,Genre_Western
0,1,Toy Story (1995),Animation|Children's|Comedy,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),Adventure|Children's|Fantasy,0,1,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),Comedy|Romance,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
3,4,Waiting to Exhale (1995),Comedy|Drama,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Father of the Bride Part II (1995),Comedy,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5,6,Heat (1995),Action|Crime|Thriller,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
6,7,Sabrina (1995),Comedy|Romance,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
7,8,Tom and Huck (1995),Adventure|Children's,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,9,Sudden Death (1995),Action,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,10,GoldenEye (1995),Action|Adventure|Thriller,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


A useful recipe for statistical applications is to combine pandas.get_dummies with a discretization function like pandas.cut:

In [22]:
np.random.seed(12345) # to make the example repeatable
values = np.random.uniform(size=10)
values

array([0.92961609, 0.31637555, 0.18391881, 0.20456028, 0.56772503,
       0.5955447 , 0.96451452, 0.6531771 , 0.74890664, 0.65356987])

In [23]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]

In [24]:
pd.get_dummies(pd.cut(values, bins))

,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,0,0,0,0,1
1,0,1,0,0,0
2,1,0,0,0,0
3,0,1,0,0,0
4,0,0,1,0,0
5,0,0,1,0,0
6,0,0,0,0,1
7,0,0,0,1,0
8,0,0,0,1,0
9,0,0,0,1,0


## String Manipulation
Python has long been a popular raw data manipulation language in part due to its ease of use for string and text processing. Most text operations are made simple with the string object’s built-in methods. For more complex pattern matching and text manipulations, regular expressions may be needed. pandas adds to the mix by enabling you to apply string and regular expressions concisely on whole arrays of data, additionally handling the annoyance of missing data.

### Python Built-In String Object Methods
In many string munging and scripting applications, built-in string methods are sufficient. As an example, a comma-separated string can be broken into pieces with split:

In [25]:
val = "a,b,  guido"
val.split(",")

['a', 'b', '  guido']

split is often combined with strip to trim whitespace (including line breaks):



In [26]:
pieces = [x.strip() for x in val.split(",")]
pieces

['a', 'b', 'guido']

These substrings could be concatenated together with a two-colon delimiter using addition:



In [27]:
first, second, third = pieces
first + "::" + second + "::" + third

'a::b::guido'

But this isn’t a practical generic method. A faster and more Pythonic way is to pass a list or tuple to the join method on the string "::":

In [31]:
"::".join(pieces)

'a::b::guido'

Other methods are concerned with locating substrings. Using Python’s in keyword is the best way to detect a substring, though index and find can also be used:

In [32]:
val

'a,b,  guido'

In [33]:
"guido" in val

True

In [34]:
val.index(",")

1

In [38]:
val.find(":")

-1

Note that the difference between find and index is that index raises an exception if the string isn’t found (versus returning –1):

In [36]:
val.index(":")

ValueError: substring not found

Relatedly, count returns the number of occurrences of a particular substring:



In [39]:
val.count(",")

2

replace will substitute occurrences of one pattern for another. It is commonly used to delete patterns, too, by passing an empty string:

In [40]:
val.replace(",", "::")


'a::b::  guido'

In [41]:
val.replace(",", "")

'ab  guido'

See Table 7.4 for a listing of some of Python's string methods.

Regular expressions can also be used with many of these operations, as you’ll see.

# ![title](i73.png)

### Regular Expressions
Regular expressions provide a flexible way to search or match (often more complex) string patterns in text. A single expression, commonly called a regex, is a string formed according to the regular expression language. Python’s built-in re module is responsible for applying regular expressions to strings; I’ll give a number of examples of its use here.

The re module functions fall into three categories: **pattern matching, substitution, and splitting**. Naturally these are all related; a regex describes a pattern to locate in the text, which can then be used for many purposes. Let’s look at a simple example: suppose we wanted to split a string with a variable number of whitespace characters (tabs, spaces, and newlines).

The regex describing one or more whitespace characters is \s+:

In [42]:
import re
text = "foo    bar\t baz  \tqux"
text

'foo    bar\t baz  \tqux'

In [43]:
re.split(r"\s+", text)

['foo', 'bar', 'baz', 'qux']

The \s (lowercase s) matches a whitespace (blank, tab \t, and newline \r or \n). On the other hand, the \S+ (uppercase S) matches anything that is NOT matched by \s, i.e., non-whitespace. In regex, the uppercase metacharacter denotes the inverse of the lowercase counterpart, for example, \w for word character and \W for non-word character; \d for digit and \D or non-digit.

An 'r' before a string tells the Python interpreter to treat backslashes as a literal (raw) character. Normally, Python uses backslashes as escape characters.


When you call re.split(r"\s+", text), the regular expression is first compiled, and then its split method is called on the passed text. You can compile the regex yourself with re.compile, forming a reusable regex object:

In [45]:
regex = re.compile(r"\s+")


In [46]:
regex.split(text)

['foo', 'bar', 'baz', 'qux']

If, instead, you wanted to get a list of all patterns matching the regex, you can use the findall method:

In [47]:
regex.findall(text)

['    ', '\t ', '  \t']

Creating a regex object with re.compile is highly recommended if you intend to apply the same expression to many strings; doing so will save CPU cycles.

match and search are closely related to findall. While findall returns all matches in a string, search returns only the first match. More rigidly, match only matches at the beginning of the string. As a less trivial example, let’s consider a block of text and a regular expression capable of identifying most email addresses:

In [49]:
text = """Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com"""
pattern = r"[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}"

# re.IGNORECASE makes the regex case insensitive
regex = re.compile(pattern, flags=re.IGNORECASE)

Using findall on the text produces a list of the email addresses:



In [50]:
regex.findall(text)

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

search returns a special match object for the first email address in the text. For the preceding regex, the match object can only tell us the start and end position of the pattern in the string:

In [51]:
m = regex.search(text)
m


<re.Match object; span=(5, 20), match='dave@google.com'>

In [52]:
text[m.start():m.end()]

'dave@google.com'

regex.match returns None, as it will match only if the pattern occurs at the start of the string:



In [53]:
print(regex.match(text))

None


Relatedly, sub will return a new string with occurrences of the pattern replaced by a new string:



In [54]:
print(regex.sub("REDACTED", text))

Dave REDACTED
Steve REDACTED
Rob REDACTED
Ryan REDACTED


Suppose you wanted to find email addresses and simultaneously segment each address into its three components: username, domain name, and domain suffix. To do this, put parentheses around the parts of the pattern to segment:

In [55]:
pattern = r"([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})"
regex = re.compile(pattern, flags=re.IGNORECASE)

A match object produced by this modified regex returns a tuple of the pattern components with its groups method:

In [57]:
m = regex.match("wesm@bright.net")
m.groups()

('wesm', 'bright', 'net')

findall returns a list of tuples when the pattern has groups:



In [58]:
regex.findall(text)

[('dave', 'google', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'yahoo', 'com')]

sub also has access to groups in each match using special symbols like \1 and \2. The symbol \1 corresponds to the first matched group, \2 corresponds to the second, and so forth:

In [59]:
print(regex.sub(r"Username: \1, Domain: \2, Suffix: \3", text))

Dave Username: dave, Domain: google, Suffix: com
Steve Username: steve, Domain: gmail, Suffix: com
Rob Username: rob, Domain: gmail, Suffix: com
Ryan Username: ryan, Domain: yahoo, Suffix: com


There is much more to regular expressions in Python, most of which is outside the book’s scope. Table 7.5 provides a brief summary.
# ![title](i74.png)


### String Functions in pandas
Cleaning up a messy dataset for analysis often requires a lot of string manipulation. To complicate matters, a column containing strings will sometimes have missing data:



In [60]:
data = {"Dave": "dave@google.com", "Steve": "steve@gmail.com",
        "Rob": "rob@gmail.com", "Wes": np.nan}
data = pd.Series(data)
data


Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                  NaN
dtype: object

In [61]:
data.isna()

Dave     False
Steve    False
Rob      False
Wes       True
dtype: bool

String and regular expression methods can be applied (passing a lambda or other function) to each value using data.map, but it will fail on the NA (null) values. To cope with this, Series has array-oriented methods for string operations that skip over and propagate NA values. These are accessed through Series’s str attribute; for example, we could check whether each email address has "gmail" in it with str.contains:

In [62]:
data.str.contains("gmail")

Dave     False
Steve     True
Rob       True
Wes        NaN
dtype: object

Note that the result of this operation has an object dtype. pandas has extension types that provide for specialized treatment of strings, integers, and Boolean data which until recently have had some rough edges when working with missing data:

In [63]:
data_as_string_ext = data.astype('string')
data_as_string_ext


Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                 <NA>
dtype: string

In [64]:
data_as_string_ext.str.contains("gmail")

Dave     False
Steve     True
Rob       True
Wes       <NA>
dtype: boolean

Extension types are discussed in more detail in Extension Data Types.

Regular expressions can be used, too, along with any re options like IGNORECASE:

In [65]:
pattern = r"([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})"
data.str.findall(pattern, flags=re.IGNORECASE)

Dave     [(dave, google, com)]
Steve    [(steve, gmail, com)]
Rob        [(rob, gmail, com)]
Wes                        NaN
dtype: object

There are a couple of ways to do vectorized element retrieval. Either use str.get or index into the str attribute:

In [66]:
matches = data.str.findall(pattern, flags=re.IGNORECASE).str[0]
matches


Dave     (dave, google, com)
Steve    (steve, gmail, com)
Rob        (rob, gmail, com)
Wes                      NaN
dtype: object

In [67]:
matches.str.get(1)

Dave     google
Steve     gmail
Rob       gmail
Wes         NaN
dtype: object

You can similarly slice strings using this syntax:



In [68]:
data.str[:5]

Dave     dave@
Steve    steve
Rob      rob@g
Wes        NaN
dtype: object

The str.extract method will return the captured groups of a regular expression as a DataFrame:



In [69]:
data.str.extract(pattern, flags=re.IGNORECASE)

The history saving thread hit an unexpected error (OperationalError('disk I/O error')).History will not be written to the database.


,0,1,2
Dave,dave,google,com
Steve,steve,gmail,com
Rob,rob,gmail,com
Wes,NaN,NaN,NaN


See Table 7.6 for more pandas string methods.

# ![title](i761.png)
# ![title](i762.png)

## Categorical Data
This section introduces the pandas Categorical type. I will show how you can achieve better performance and memory use in some pandas operations by using it. I also introduce some tools that may help with using categorical data in statistics and machine learning applications.

### Background and Motivation
Frequently, a column in a table may contain repeated instances of a smaller set of distinct values. We have already seen functions like unique and value_counts, which enable us to extract the distinct values from an array and compute their frequencies, respectively:

In [ ]:
values = pd.Series(['apple', 'orange', 'apple',
                    'apple'] * 2)
values
pd.unique(values)
pd.value_counts(values)

In [ ]:
values = pd.Series([0, 1, 0, 0] * 2)
dim = pd.Series(['apple', 'orange'])
values
dim

In [ ]:
dim.take(values)

In [ ]:
fruits = ['apple', 'orange', 'apple', 'apple'] * 2
N = len(fruits)
rng = np.random.default_rng(seed=12345)
df = pd.DataFrame({'fruit': fruits,
                   'basket_id': np.arange(N),
                   'count': rng.integers(3, 15, size=N),
                   'weight': rng.uniform(0, 4, size=N)},
                  columns=['basket_id', 'fruit', 'count', 'weight'])
df

In [ ]:
fruit_cat = df['fruit'].astype('category')
fruit_cat

In [ ]:
c = fruit_cat.array
type(c)

In [ ]:
c.categories
c.codes

In [ ]:
dict(enumerate(c.categories))

In [ ]:
df['fruit'] = df['fruit'].astype('category')
df["fruit"]

In [ ]:
my_categories = pd.Categorical(['foo', 'bar', 'baz', 'foo', 'bar'])
my_categories

In [ ]:
categories = ['foo', 'bar', 'baz']
codes = [0, 1, 2, 0, 0, 1]
my_cats_2 = pd.Categorical.from_codes(codes, categories)
my_cats_2

In [ ]:
ordered_cat = pd.Categorical.from_codes(codes, categories,
                                        ordered=True)
ordered_cat

In [ ]:
my_cats_2.as_ordered()

In [ ]:
rng = np.random.default_rng(seed=12345)
draws = rng.standard_normal(1000)
draws[:5]

In [ ]:
bins = pd.qcut(draws, 4)
bins

In [ ]:
bins = pd.qcut(draws, 4, labels=['Q1', 'Q2', 'Q3', 'Q4'])
bins
bins.codes[:10]

In [ ]:
bins = pd.Series(bins, name='quartile')
results = (pd.Series(draws)
           .groupby(bins)
           .agg(['count', 'min', 'max'])
           .reset_index())
results

In [ ]:
results['quartile']

In [ ]:
N = 10_000_000
labels = pd.Series(['foo', 'bar', 'baz', 'qux'] * (N // 4))

In [ ]:
categories = labels.astype('category')

In [ ]:
labels.memory_usage(deep=True)
categories.memory_usage(deep=True)

In [ ]:
%time _ = labels.astype('category')

In [ ]:
%timeit labels.value_counts()
%timeit categories.value_counts()

In [ ]:
s = pd.Series(['a', 'b', 'c', 'd'] * 2)
cat_s = s.astype('category')
cat_s

In [ ]:
cat_s.cat.codes
cat_s.cat.categories

In [ ]:
actual_categories = ['a', 'b', 'c', 'd', 'e']
cat_s2 = cat_s.cat.set_categories(actual_categories)
cat_s2

In [ ]:
cat_s.value_counts()
cat_s2.value_counts()

In [ ]:
cat_s3 = cat_s[cat_s.isin(['a', 'b'])]
cat_s3
cat_s3.cat.remove_unused_categories()

In [ ]:
cat_s = pd.Series(['a', 'b', 'c', 'd'] * 2, dtype='category')

In [ ]:
pd.get_dummies(cat_s, dtype=float)

In [ ]:
pd.options.display.max_rows = PREVIOUS_MAX_ROWS